In [1]:
# Cálcula el factor de reducción de área para datos en malla.

import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt

In [57]:
# Cargamos el archivo.
fdir = "../data/CHIRPS_megalopolis.nc"
ds_tot = xr.open_dataset( fdir )

# Ajustamos nombres.
ds_tot = ds_tot.rename_vars( { "longitude": "LONGITUD", "latitude": "LATITUD",
    "time": "TIEMPO", "precip": "PRECIPITACION" } ).swap_dims(
    { "longitude": "LONGITUD", "latitude": "LATITUD", "time": "TIEMPO" } 
    ).to_dataframe().to_xarray()

ds_tot

<xarray.Dataset>
Dimensions:        (TIEMPO: 15340, LONGITUD: 64, LATITUD: 52)
Coordinates:
  * TIEMPO         (TIEMPO) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * LONGITUD       (LONGITUD) float64 -100.5 -100.4 -100.4 ... -97.38 -97.33
  * LATITUD        (LATITUD) float64 18.12 18.17 18.22 ... 20.57 20.62 20.67
Data variables:
    PRECIPITACION  (TIEMPO, LONGITUD, LATITUD) float32 0.0 0.0 0.0 ... 0.0 0.0

In [93]:
# Cálculo de ARF.

pre = "PRECIPITACION"
#dur = [1, 2,  3,  4,  5, 10, 15, 20, 30, 40, 50, 60]
dur = [1]

ds_trt = []

# Cantidad máxima de celdas a quitar de la orilla.
j_a = int( min( ds_tot["LONGITUD"].shape[0] / 2 - 1,
           ds_tot["LATITUD" ].shape[0] / 2 - 1 ) )

# Iteramos para áreas decrecientes.
for j in range( 0, j_a + 1 ):
    # Seleccionamos el área de estudio.
    ds_i = ds_tot.isel(
        { "LONGITUD": slice( j, ds_tot["LONGITUD"].shape[0] - j ),
           "LATITUD": slice( j, ds_tot["LATITUD" ].shape[0] - j ) } )

    # Cantidad de años de estudio.
    years = ds_i["TIEMPO"].dt.year.to_dataframe()["year"].unique()
    ny = years.shape[0]

    # SAM para cada celda.
    ds_tret = []
    # Iteramos para cada duración.
    for i in range( len(dur) ):
        # Creamos un dataset para colocar la SAM.
        ds_y = ds_i.isel( {"TIEMPO": range(0, ny) } ).copy()
        ds_tret.append( ds_y )
        ds_tret[i]["TIEMPO"] = years
        ds_p = ds_tret[i]["PRECIPITACION"]

        # Obtenemos la precipitación acumulada para la duración.
        ds_t = ds_i.rolling( {"TIEMPO": dur[i]} ).sum()
        # Obtenemos el máximo para cada año.
        for y in years:
            ds_p[ ds_p["TIEMPO"] == y ] = ( ds_t[pre].sel(
                { "TIEMPO": (ds_t["TIEMPO"].dt.year == y) }
                ).max(dim = "TIEMPO") )

        # Iteramos para cada celda.
        for lon in ds_tret[i]["LONGITUD"]:
            for lat in ds_tret[i]["LATITUD"]:
                # Ordenamos la SAM.
                df_p = ds_p[ :, ds_p["LONGITUD"] == lon,
                    ds_p["LATITUD"] == lat ].to_dataframe().sort_values( pre
                    ).reset_index().rename( {"TIEMPO": "AÑO"}, axis = 1 )
                df_p["TIEMPO"] = years
                df_p = df_p.set_index( ["TIEMPO", "LONGITUD", "LATITUD"] )
                # Asignamos los valores ordenados.
                ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat ] = (
                    df_p.to_xarray()[pre] )

        # Calculamos el tiempo de retorno.
        ds_tret[i]["TIEMPO"] = ( ny + 1 ) / np.arange(ny, 0, -1)
        ds_tret[i] = ds_tret[i].rename_dims( {"TIEMPO": "TIEMPO_RETORNO"} 
                              ).rename_vars( {"TIEMPO": "TIEMPO_RETORNO"} )
    
        # Promediamos la SAM de todas las celdas.
        ds_tret[i] = ds_tret[i].mean( ["LONGITUD", "LATITUD"]
            ).expand_dims( {"DURACION": [dur[i]] } )

    ds_tret = xr.concat(ds_tret, "DURACION")

    # SAM para la precipitación media de la zona de estudio.
    df_i = ds_i.mean(
        ["LONGITUD", "LATITUD"] ).to_dataframe()
    df_i["CURVA_MASA"] = df_i["PRECIPITACION"].cumsum()

    # Iteramos para cada duración.
    for i in dur:
        # Obtenemos la precipitación acumulada para la duración.
        df_i[ i ] = ( ( df_i["CURVA_MASA"] - df_i["CURVA_MASA"].shift(i) ) )
    df_i.columns.name = "DURACION"
    # Obtenemos el máximo para cada año.
    df_i = df_i.drop( ["PRECIPITACION", "CURVA_MASA"],
        axis = 1 ).groupby( df_i.index.year ).max()
    # Ordenamos la SAM.
    df_i = df_i.apply( lambda x: x.sort_values(
        ascending = False ).values, axis = 0 )
    
    # Calculamos el tiempo de retorno.
    df_i = df_i.reset_index( drop = True )
    df_i.index.name = "TIEMPO_RETORNO"
    df_i.index += 1
    df_i.index = ( df_i.index[-1] + 1 ) / df_i.index
    df_i = df_i.sort_index()

    # Calculamos el ARF.
    ds_tret = df_i.stack().to_xarray() / ds_tret

    # Calculamos el área de estudio.
    ds_trt.append( ds_tret.expand_dims( { "AREA": [ (
        ds_i["LONGITUD"].shape[0] * 5 * ds_i["LATITUD"].shape[0] * 5 ) ] } ) )

ds_trt = xr.concat(ds_trt, "AREA")

ds_trt

,,TIEMPO_RETORNO,PRECIPITACION
AREA,DURACION,,
83200,1,43.0,0.497283
77500,1,43.0,0.503895
72000,1,43.0,0.510743
66700,1,43.0,0.517004
61600,1,43.0,0.523520
56700,1,43.0,0.531581
52000,1,43.0,0.539046
47500,1,43.0,0.547299
43200,1,43.0,0.554705


In [102]:
ds_trt.to_dataframe().droplevel("DURACION").unstack()

PRECIPITACION                                          \
TIEMPO_RETORNO     1.023810  1.048780  1.075000  1.102564  1.131579    
AREA                                                                   
83200               0.605463  0.567964  0.599059  0.600494  0.598892   
77500               0.605379  0.560194  0.605062  0.588412  0.607344   
72000               0.588930  0.578323  0.605537  0.585975  0.615503   
66700               0.573706  0.588268  0.603317  0.585844  0.614675   
61600               0.557379  0.598972  0.598532  0.585229  0.601092   
56700               0.539603  0.608383  0.590784  0.581019  0.590043   
52000               0.522395  0.610804  0.583854  0.573741  0.591614   
47500               0.521034  0.599096  0.590620  0.581078  0.598869   
43200               0.522545  0.631660  0.598783  0.589987  0.606471   
39100               0.523101  0.651752  0.626773  0.609834  0.622261   
35200               0.526908  0.673951  0.643853  0.644335  0.649275   
31500               0.529396  0.699822  0.667737  0.661498  0.653850   
28000               0.532589  0.734924  0.700732  0.685511  0.679984   
24700               0.550486  0.764273  0.733499  0.712008  0.694895   
21600               0.575179  0.772097  0.753732  0.735587  0.724746   
18700               0.599445  0.766953  0.770018  0.745993  0.733016   
16000               0.632389  0.765867  0.775686  0.750335  0.732303   
13500               0.658717  0.762526  0.773509  0.746076  0.740358   
11200               0.690695  0.760295  0.764126  0.743039  0.749876   
9100                0.733586  0.786375  0.751075  0.740816  0.756134   
7200                0.763246  0.773754  0.760033  0.738859  0.758176   
5500                0.774407  0.760983  0.733856  0.751854  0.759680   
4000                0.780182  0.747567  0.735762  0.754238  0.781280   
2700                0.775839  0.743820  0.770485  0.825309  0.830810   
1600                0.788680  0.781594  0.783896  0.863863  0.856117   
700                 0.838630  0.805783  0.773810  0.829087  0.873075   

                                                                  ...  \
TIEMPO_RETORNO 1.162162  1.194444  1.228571  1.264706  1.303030   ...   
AREA                                                              ...   
83200           0.582310  0.582526  0.574538  0.566755  0.559987  ...   
77500           0.602328  0.590001  0.577937  0.574834  0.568834  ...   
72000           0.610158  0.596647  0.583342  0.579410  0.571470  ...   
66700           0.606118  0.599835  0.588174  0.591787  0.586230  ...   
61600           0.612884  0.605955  0.594138  0.599598  0.591421  ...   
56700           0.619314  0.613565  0.600034  0.615329  0.604674  ...   
52000           0.626146  0.619737  0.610221  0.627322  0.620105  ...   
47500           0.634662  0.627589  0.617885  0.639564  0.630406  ...   
43200           0.644768  0.636430  0.626691  0.649098  0.638450  ...   
39100           0.663000  0.650030  0.642931  0.635583  0.643964  ...   
35200           0.645284  0.650488  0.647053  0.657750  0.653525  ...   
31500           0.663344  0.649787  0.662995  0.655230  0.655739  ...   
28000           0.666685  0.678362  0.674586  0.662681  0.657935  ...   
24700           0.692482  0.695676  0.681487  0.668970  0.657625  ...   
21600           0.708637  0.700154  0.688896  0.674718  0.662931  ...   
18700           0.718627  0.709522  0.697080  0.683781  0.683530  ...   
16000           0.738072  0.722823  0.711226  0.700130  0.701980  ...   
13500           0.733144  0.727389  0.729263  0.717442  0.722720  ...   
11200           0.732085  0.735755  0.744855  0.734958  0.723074  ...   
9100            0.738206  0.753760  0.748882  0.746542  0.744404  ...   
7200            0.761860  0.781874  0.765739  0.766820  0.765644  ...   
5500            0.789919  0.788180  0.785149  0.788197  0.778270  ...   
4000            0.789748  0.797041  0.793664  0.781520  0.777168  ...   
2700            0.814194  0.